# OPTIONS

In [ ]:
score_type = 'EUC'  # COS

# SETUP

In [ ]:
!git clone https://github.com/Yichuan0712/rla.git

Cloning into 'rla'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 332 (delta 59), reused 81 (delta 28), pack-reused 209
Receiving objects: 100% (332/332), 39.95 MiB | 35.29 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [ ]:
import os
os.chdir('rla')
os.listdir(os.getcwd())

['yichuan_colab',
 'example_data',
 'terminator_utils',
 'src',
 '__pycache__',
 'dataset_configs',
 'contact_head',
 'terminator_configs',
 'example_notebooks',
 'pdb_blacklist.txt',
 'terminator',
 '.idea',
 '.git',
 'rla_env.yml',
 'clip_main.py',
 'README.md',
 'images']

In [ ]:
!pip install open_clip_torch

import torch
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

!pip install fastargs
!pip install terminaltables
!pip install biopython
!pip install webdataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x8

In [ ]:
import os
import src.models_and_optimizers as model_utils
import yaml
from types import SimpleNamespace
from clip_main import get_wds_loaders
from transformers import EsmTokenizer
import src.data_utils as data_utils
import torch
import sys
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import json
from torch.cuda.amp import autocast
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import webdataset as wds
import copy

<ipython-input-5-264f5a970416>:16: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [ ]:
## Load util functions
# os.chdir('example_notebooks')
!pip install tmtools
!pip install tmscoring
!pip install iminuit
# %run example_notebooks/rla_utils.ipynb
# os.chdir('..')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tmscoring: filename=tmscoring-0.4.post0-py3-none-any.whl size=5755 sha256=e1644de91677fe907021e969aa236ff12ebba3707ad179c35caae09394424c64
  Stored in directory: /root/.cache/pip/wheels/42/7f/85/dd37516a1a64921564b4a7cb3e5640eadecb624c215ded2480
Successfully built tmscoring
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 5.2 MB/s eta 0:00:00


In [ ]:

import os
import src.models_and_optimizers as model_utils
import yaml
from types import SimpleNamespace
from clip_main import get_wds_loaders
from transformers import EsmTokenizer
import src.data_utils as data_utils
import torch
import sys
import pickle
from tqdm import tqdm
import numpy as np
from tmtools.io import get_structure, get_residue_data
from tmtools import tm_align
import matplotlib.pyplot as plt
import json
from torch.cuda.amp import autocast
import tmscoring
import json
import copy
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import yaml
import pandas as pd
import glob
import webdataset as wds

/usr/local/lib/python3.10/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
<ipython-input-7-c45d07920da9>:21: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git pull origin main

From https://github.com/Yichuan0712/rla
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
!cp /content/drive/MyDrive/RLA -r /content/rla

# rla_utls

In [ ]:
## Mask peptide sequence information
def mask_peptide(seq_batch, coords_batch, pdb):
    chain_len_dicts = {}
    lens = coords_batch['seq_lens']
    chain_len_dicts['protein'] = max(lens)
    chain_len_dicts['peptide'] = min(lens)
    from_back = lens[1] == min(lens)
    if from_back:
        seq_batch['string_sequence'][0] = seq_batch['string_sequence'][0][:-peptide_len] + 'X'*peptide_len
        seq_batch['seq_loss_mask'][0][:,:-1*peptide_len] = False
        seq_batch['seq_loss_mask'][1][:,:-1*peptide_len] = False
    else:
        seq_batch['string_sequence'][0] = 'X'*peptide_len + seq_batch['string_sequence'][0][peptide_len:]
        seq_batch['seq_loss_mask'][0][:,peptide_len:] = False
        seq_batch['seq_loss_mask'][1][:,peptide_len:] = False
    return seq_batch

def mask_all(seq_batch, pdb):
    seq_batch['string_sequence'][0] = 'X'*len(seq_batch['string_sequence'][0])
    seq_batch['seq_loss_mask'][0][:,:] = False
    seq_batch['seq_loss_mask'][1][:,:] = False
    return seq_batch

## Extract kNN info
def extract_knn(X, eps, top_k):
    # Convolutional network on NCHW
    dX = torch.unsqueeze(X, 1) - torch.unsqueeze(X, 2)
    D = torch.sqrt(torch.sum(dX**2, 3) + eps)
    mask_2D = torch.ones(D.shape)
    D *= mask_2D

    # Identify k nearest neighbors (including self)
    D_max, _ = torch.max(D, -1, keepdim=True)
    D_adjust = D + (1. - mask_2D) * D_max
    D_neighbors, E_idx = torch.topk(D_adjust, top_k, dim=-1, largest=False)
    return D_neighbors, E_idx

## Find residues at the interface between 2 chains
def get_interaction_res(coords_batch, pdb, top_k, threshold=5, remove_far=False, mres=None, dist_threshold=None):
    chain_len_dicts = {}
    lens = coords_batch['seq_lens'][0]
    chain_len_dicts['protein'] = max(lens)
    chain_len_dicts['peptide'] = min(lens)
    if len(lens) > 1:
        from_back = lens[1] == min(lens)
    else:
        from_back = True
    top_k = min(top_k, coords_batch['coords'][0].size(1))
    D_neighbors, E_idx = extract_knn(coords_batch['coords'][0][:,:,1,:], eps=1e-6, top_k=top_k)
    if from_back:
        interaction_res = set(range(chain_len_dicts['protein'], chain_len_dicts['protein']+chain_len_dicts['peptide']))
    else:
        interaction_res = set(range(0, chain_len_dicts['peptide']))
    # interaction_res = set(range(0, chain_len_dicts['protein']+chain_len_dicts['peptide']))
    if mres is not None:
        interaction_res = set(mres)
    if top_k == 0:
        return list(interaction_res)
    prot_to_add = set()
    for res in interaction_res:
        neighs_to_add = list()
        for dist, neigh in zip(D_neighbors[0, res], E_idx[0, res]):
            if dist_threshold and dist > dist_threshold:
                continue
            neighs_to_add.append(neigh)
        prot_to_add = prot_to_add.union(set(neighs_to_add))
    interaction_res = list(interaction_res.union(prot_to_add))
    if remove_far:
        to_remove = []
        chain_lens = torch.cat([torch.zeros(lens[0]), torch.ones(lens[1])])
        for res in interaction_res:
            nother = 0

            opp = 1 - chain_lens[res].item()
            for nres in E_idx[0, res]:
                if chain_lens[nres].item() == opp:
                    nother += 1

            if nother < threshold:
                to_remove.append(res)
        for res in to_remove:
            interaction_res.remove(res)
    return interaction_res

## Get distances between residues at the interface between 2 chains
def get_inter_dists(coords_batch, interaction_res, eps=1e-6):
    chain_len_dicts = {}
    chains, lens = torch.unique_consecutive(coords_batch['chain_lens'][0][0], return_counts=True)
    chain_len_dicts['protein'] = torch.max(lens)
    chain_len_dicts['peptide'] = torch.min(lens)
    from_back = torch.argmin(lens) == 1
    if from_back:
        pep_res = list(range(chain_len_dicts['protein'], chain_len_dicts['protein']+chain_len_dicts['peptide']))
        prot_res = ppe_res = list(range(0, chain_len_dicts['protein']))
    else:
        pep_res = list(range(0, chain_len_dicts['peptide']))
        prot_res = list(range(chain_len_dicts['peptide'], chain_len_dicts['peptide']+chain_len_dicts['protein']))
    pep_coords = coords_batch['coords'][0][:,pep_res,1,:]
    prot_coords = coords_batch['coords'][0][:,prot_res,1,:]
    inter_dists = []
    for res in range(chain_len_dicts['protein'] + chain_len_dicts['peptide']):
        if not res in interaction_res:
            inter_dists.append(np.nan)
            continue
        res_coord = coords_batch['coords'][0][:,res,1,:]
        if res in pep_res:
            dists = torch.sum((res_coord - prot_coords)**2, dim=2)
        else:
            dists = torch.sum((res_coord - pep_coords)**2, dim=2)
        inter_dists.append(torch.min(dists))
    inter_dists = np.array(inter_dists)
    inter_dists = inter_dists / np.nanmax(inter_dists)
    inter_dists = np.nan_to_num(inter_dists)
    return inter_dists

## Mask the structure of the peptide
def mask_peptide_struct(coord_data, coords_batch, pdb):
    peptide_res = get_interaction_res(coords_batch, pdb, top_k=0)
    protein_mask = torch.ones(coord_data['x_mask'].shape).to(dtype=coord_data['x_mask'].dtype, device=coord_data['x_mask'].device)
    protein_mask[:,peptide_res] = 0
    coord_data['x_mask'] *= protein_mask
    coord_data['X'] *= protein_mask.unsqueeze(-1).unsqueeze(-1)
    return coord_data

## Get sequence and structure embeddings from RLA
def get_seq_and_struct_features(model, tokenizer, batch, pdb=None, seq_mask=None, struct_mask=None, focus=None, top_k=30, remove_far=False, mres=None,
                                prot_len=None, add_ends=False, ends_k=0, threshold=1, seq_only=False, dist_threshold=None):
    seq_batch, coords_batch = batch
    if prot_len is not None:
        seq_batch['string_sequence'][0] = [seq_batch['string_sequence'][0][:prot_len] + 25*'G' + seq_batch['string_sequence'][0][prot_len:]]
        base_len = len(seq_batch['string_sequence'][0])
        pos_embs = torch.arange(base_len+25).unsqueeze(0)
        seq_mask = torch.ones(base_len+25).to(dtype=torch.bool).unsqueeze(0)
        seq_batch['pos_embs'] = [pos_embs, seq_mask]
        seq_batch['placeholder_mask'] = [seq_mask, seq_mask]
        seq_batch['seq_loss_mask'] = [seq_mask, seq_mask]
    if add_ends:
        base_len = len(seq_batch['string_sequence'][0])
        seq_batch['string_sequence'][0] = ends_k*'X' + seq_batch['string_sequence'][0] + ends_k*'X'
        pos_embs = torch.arange(base_len+2*ends_k).unsqueeze(0)
        seq_mask = torch.zeros(base_len+2*ends_k)
        seq_mask[ends_k:-ends_k] = 1
        seq_mask = seq_mask.to(dtype=torch.bool).unsqueeze(0)
        seq_batch['pos_embs'] = [pos_embs, seq_mask]
        seq_batch['placeholder_mask'] = [seq_mask, seq_mask]
        seq_batch['seq_loss_mask'] = [seq_mask, seq_mask]
    if seq_mask == 'peptide':
        seq_batch = mask_peptide(seq_batch, coords_batch, pdb)
    if seq_mask == 'all':
        seq_batch = mask_all(seq_batch, pdb)
    seqs = seq_batch['string_sequence']
    text_inp = tokenizer(seqs, return_tensors='pt', padding=True, truncation=True, max_length=1024+2)
    text_inp['position_ids'] = seq_batch['pos_embs'][0]
    text_inp = {k: v.to('cuda') for k, v in text_inp.items()}
    if not seq_only:
        coord_data = data_utils.construct_gnn_inp(coords_batch, device='cuda', half_precision=True)
        if struct_mask=='peptide':
            coord_data = mask_peptide_struct(coord_data, coords_batch, pdb)
        gnn_features, text_features, logit_scale = model(text_inp, coord_data)
    else:
        gnn_features, text_features, logit_scale = model(text_inp, None)
    new_text_features, _, new_text_mask = data_utils.postprocess_text_features(
        text_features=text_features,
        inp_dict=text_inp,
        tokenizer=tokenizer,
        placeholder_mask=seq_batch['placeholder_mask'][0])
    if prot_len is not None:
        new_text_features = torch.cat((new_text_features[:,:prot_len,:], new_text_features[:,prot_len+25:,:]), dim=1)
        new_text_mask = torch.cat((new_text_mask[:,:prot_len], new_text_mask[:,prot_len+25:]), dim=1)
    if focus:
        focus_res = get_interaction_res(coords_batch, pdb, top_k, remove_far = remove_far, threshold = threshold, mres = mres, dist_threshold = dist_threshold)
        focus_mask = torch.zeros(coords_batch['coords'][1].shape).to(dtype=coords_batch['coords'][1].dtype, device=new_text_features.device)
        focus_mask[:,focus_res] = True
    else:
        focus_mask = torch.zeros(coords_batch['coords'][1].shape).to(dtype=coords_batch['coords'][1].dtype, device=new_text_features.device)
    return {
        'text': new_text_features, # text feature
        'gnn': gnn_features, # gnn feature
        'seq_mask_with_burn_in': seq_batch['seq_loss_mask'][0], # sequence mask of what's supervised
        'coord_mask_with_burn_in': coords_batch['coords_loss_mask'][0], # coord mask of what's supervised
        'seq_mask_no_burn_in': new_text_mask.bool(), # sequence mask of what's valid (e.g., not padded)
        'coord_mask_no_burn_in': coords_batch['coords'][1], # coord mask of what's valid
        'focus_mask': focus_mask, # focus mask of what residues to use to calculate score
    }

## Calculate RLA score
cos = torch.nn.CosineSimilarity()
def calc_sim(all_outputs):
    all_sims = []
    all_sims_burn = []
    for output in all_outputs:
        t = output['text'][output['seq_mask_no_burn_in']]
        g = output['gnn'][output['coord_mask_no_burn_in']]
        print('t', t.shape)
        print('g', g.shape)
        sim = (t.unsqueeze(1) @ g.unsqueeze(-1)).squeeze(1).squeeze(1)
        all_sims.append(torch.mean(sim))

    return all_sims

# Get pdb ids of proteins in a batch
def get_decoy_ids(wds_path):
    cols = ['inp.pyd']
    wd_ds = wds.WebDataset(wds_path).decode().to_tuple(*cols)
    batched_ds = wd_ds.batched(1, collation_fn=loaders_utils.custom_collation_fn)
    decoy_ids = []
    for i, b in enumerate(wd_ds):
        decoy_ids.append(b[0]['pdb_id'][0])
    return decoy_ids

In [ ]:
## ESM mutation analysis functions
def get_muts(wt, mut):
    inds = []
    muts = []
    for i, (wchar, mchar) in enumerate(zip(wt, mut)):
        if wchar != mchar:
            inds.append(i)
            muts.append(mchar)
    return inds, muts

def score_mut(wt, idx, mt, token_probs, alphabet):
    wt = wt[idx]
    wt_encoded, mt_encoded = alphabet.get_idx(wt), alphabet.get_idx(mt)

    # add 1 for BOS
    score = token_probs[0, 1 + idx, mt_encoded] - token_probs[0, 1 + idx, wt_encoded]
    return score.item()

def score_protein(idxs, mts, wt_seq, model, alphabet):

    # inference for each model
    batch_converter = alphabet.get_batch_converter()

    data = [
        ("protein1", wt_seq),
    ]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    all_token_probs = []
    for i in tqdm(range(batch_tokens.size(1))):
        batch_tokens_masked = batch_tokens.clone()
        batch_tokens_masked[0, i] = alphabet.mask_idx
        with torch.no_grad():
            token_probs = torch.log_softmax(
                model(batch_tokens_masked.cuda())["logits"], dim=-1
            )
        all_token_probs.append(token_probs[:, i])  # vocab size
    token_probs = torch.cat(all_token_probs, dim=0).unsqueeze(0)
    esm_predictions = []
    for idx_list, mut_list in zip(idxs, mts):
        mut_score = 0
        for idx, mut in zip(idx_list, mut_list):
            mut_score += score_mut(wt_seq, idx, mut, token_probs, alphabet)
        if len(idx_list) == 0:
            mut_score = np.nan
        esm_predictions.append(mut_score)
    return esm_predictions

# START

In [ ]:
## Load model parameters

model_dir = "/content/rla/RLA"
path = os.path.join(model_dir, "checkpoints/checkpoint_best.pt")

args_path = os.path.join(model_dir,
 [u for u in os.listdir(model_dir) if u.endswith('.pt')][0])

hparams = torch.load(args_path)
hparams['args']['blacklist_file'] = "pdb_blacklist.txt"
hparams['args']['coordinator_hparams'] = "terminator_configs/standard.json"
args_dict = hparams['args']
args_dict['batch_size'] = 1
args_dict['distributed'] = 0

args_dict['train_wds_path'] = 'dataset.tar'
args_dict['val_wds_path'] = 'dataset.tar'
args_dict['data_root'] = "/content/rla/RLA/example_data"

# args_dict['train_wds_path'] = "train/shard-{000000..000019}.tar"
# args_dict['val_wds_path'] = "val/{000000..000004}.tar"
# args_dict['data_root'] = "/content/rla/RLA/pdb_wds"

args_dict['arch'] = 'facebook/esm2_t30_150M_UR50D'

args = SimpleNamespace(**args_dict)
coordinator_params = data_utils.get_coordinator_params(args.coordinator_hparams)
coordinator_params['num_positional_embeddings'] = args.gnn_num_pos_embs
coordinator_params['zero_out_pos_embs'] = args.gnn_zero_out_pos_embs
coordinator_params['clip_mode'] = True

In [ ]:
## Load model
trained_model = model_utils.load_model(path, args_dict['arch'], 'cuda')
tokenizer = EsmTokenizer.from_pretrained(args_dict['arch'])

loading state dict from /content/rla/RLA/checkpoints/checkpoint_best.pt
building model based on path


config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ZERO OUT POS EMB False
GNN Potts Model Encoder hidden dimensionality is 128
freeze_llm False
{'training_args': {'epochs': 10, 'lr': 0.001, 'weight_decay': 0.001, 'momentum': 0.9, 'label_smoothing': 0.0, 'lr_peak_epoch': 2, 'eval_epochs': 2, 'only_non_bn_weight_decay': False, 'opt': 'ADAM', 'lr_scheduler': 'cosine', 'burn_in': 0, 'mixed_precision': True, 'max_len': 1024, 'self_supervised': True}, 'epoch': 6, 'training_metrics': {'loss': 0.912731235943793, 'acc': 0.5713620556928036}, 'val_metrics': {'loss': 1.5782045675865344, 'acc': 0.5098662864254268}, 'model_building_args': {'esm_arch': 'facebook/esm2_t30_150M_UR50D', 'terminator_hparams': {'cov_features': 'all_raw', 'cov_compress': 'project', 'term_use_mpnn': True, 'matches': 'transformer', 'energies_style': 'mpnn', 'energies_use_mpnn': True, 'energies_full_graph': True, 'contact_idx': True, 'energies_encoder_layers': 3, 'energies_hidden_dim': 128, 'resnet_linear': True, 'matches_linear': True, 'transformer_linear': True, 'term_mpnn_

tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
## Load and format ground truth data
import pandas as pd
dataset_file = '/content/rla/RLA/Tsuboyama2023_Dataset2_Dataset3_20230416.csv'
stability_scores = pd.read_csv(dataset_file)
stability_scores = stability_scores[stability_scores['dG_ML']!='-'].copy(deep=True) # Drop unreliable data
stability_scores = stability_scores[stability_scores['ddG_ML']!='-'].copy(deep=True)
stability_scores['dG_ML'] = pd.to_numeric(stability_scores['dG_ML'])
stability_scores['ddG_ML'] = pd.to_numeric(stability_scores['ddG_ML'])
stability_scores = stability_scores[~stability_scores['name'].str.contains('_con')].copy(deep=True) # Drop not modeled proteins
stability_scores = stability_scores[~stability_scores['name'].str.contains('ins')].copy(deep=True) # Drop indels
stability_scores = stability_scores[~stability_scores['name'].str.contains('del')].copy(deep=True)

# stability_scores_drop = stability_scores.groupby('aa_seq').mean().reset_index()
# stability_scores_drop = stability_scores.groupby('aa_seq').mean(numeric_only=True).reset_index()
numeric_cols = stability_scores.select_dtypes(include='number').columns
stability_scores_drop = stability_scores.groupby('aa_seq')[numeric_cols].mean().reset_index()

stability_scores_drop_max = stability_scores.groupby('aa_seq').agg({'WT_name': 'max'}).reset_index()
stability_scores_drop['WT_name'] = stability_scores_drop_max['WT_name']

<ipython-input-15-fdc9d1a1c261>:4: DtypeWarning: Columns (30,31,36) have mixed types. Specify dtype option on import or set low_memory=False.
  stability_scores = pd.read_csv(dataset_file)


# NEW

In [ ]:
## Define subset of example proteins to test and define test parameters
test_subset = []
# test_subset = ['5jrt']
# test_subset = ['5jrt', '2k1b']
# test_subset = ['1pv0', '1yu5', '5jrt', '2k1b']

from scipy.stats import spearmanr
def output_results(all_scores_esm_rla_muts,outputfile):
    output=open(outputfile,'w')
    for pdb in all_scores_esm_rla_muts.keys():
        pdb_dataset = stability_scores_drop[stability_scores_drop['WT_name'] == pdb + '.pdb']
        esm_rla_df = pd.DataFrame(all_scores_esm_rla_muts[pdb])
        results_merge = esm_rla_df.merge(pdb_dataset)
        esm_rla_corr = results_merge['dG_ML'].corr(results_merge['ESM-RLA scores'])
        # esm_rla_spearmn, p_value= spearmanr(results_merge['dG_ML'], results_merge['ESM-RLA scores'])
        esm_rla_spearmn= results_merge['dG_ML'].corr(results_merge['ESM-RLA scores'],method='spearman')
        output.write(f'{pdb}\t{esm_rla_corr}\t{esm_rla_spearmn}\n')

    output.close()

## Score mutations and compare to observed values
seq_mask=False
struct_mask=False
top_k = 30
focus = True ## Focus RLA score on residues around mutations
add_ends = False
ends_k = 0
remove_far = False
seq_only = True  # 这
dist_threshold = None
feature_getter = get_seq_and_struct_features
pdb_i = 0




## 2 different scores are calculated: the RLA score of the mutant and the change in ESM-RLA embeddings from the wild-type to the mutant

all_scores = {}
all_scores_muts = {}
all_corrs = {}
all_scores_esm_rla_muts = {}

yichuan_list = []
yichuan_count = 0

train_loader, val_loader, train_len, val_len = get_wds_loaders(args, coordinator_params, gpu=None, shuffle_train=False, val_only=True, return_count=False) # Load data

for i, batch in enumerate(val_loader):
    if len(batch[0]['string_sequence'][0]) != batch[1]['coords'][0].shape[1]: # Ensure matching sequence and structure
        continue
    wt_seq = batch[0]['string_sequence'][0]
    pdb = batch[0]['pdb_id'][0]
    if len(test_subset)!=0:
      if pdb.lower() not in test_subset: # Only analyze proteins in test_subset
          continue

    batchc = copy.deepcopy(batch)
    pdb = pdb.replace(':', '|')
    # print(pdb)
    pdb_dataset = stability_scores_drop[stability_scores_drop['WT_name'] == pdb + '.pdb']
    if len(pdb_dataset) == 0: # Only analyze proteins with ground-truth data
        continue

    # Extract ground-truth data
    pdb_seqs = np.array(list(pdb_dataset['aa_seq']))
    exp_energies = -1*pdb_dataset['dG_ML'].values
    all_seq_scores = {'aa_seq': [], 'RLA res scores': []}
    all_mean_scores = {'aa_seq': [], 'RLA scores': []}
    esm_rla_scores_muts = {'aa_seq': [], 'ESM-RLA scores': []}

    with torch.no_grad():
        with autocast(dtype=torch.float16):
            # Calculate base RLA scores for WT sequence
            output_dict = feature_getter(trained_model, tokenizer, batch, pdb=None)
            text_feat = output_dict['text']
            text_feat_base = copy.deepcopy(text_feat)
            gnn_feat =  output_dict['gnn'][:, :text_feat.shape[1]] # Remove tail padding

            # print("text_feat", text_feat.shape)
            # print("gnn_feat", gnn_feat.shape)
            if score_type == 'COS':
                scores = (text_feat.unsqueeze(2) @ gnn_feat.unsqueeze(-1)).squeeze(-1).squeeze(-1)
            elif score_type == 'EUC':
                diff = text_feat - gnn_feat
                squared_diff = diff ** 2
                sum_squared_diff = torch.sum(squared_diff, dim=2)
                scores = torch.sqrt(sum_squared_diff).squeeze(0)  # [49]

            scores = (scores * output_dict['seq_mask_no_burn_in'].float()).sum(1)/output_dict['seq_mask_no_burn_in'].sum(1)
            base_score = scores.cpu()
            all_scores[pdb] = scores.cpu()
            skip_prot = False

            # Iterate over sequences and calculate RLA scores
            inds = []
            muts = []
            i_pdb_seq = 0
            i_seq = 0
            filt_pdb_seqs = []
            for pdb_seq in tqdm(pdb_seqs):
                batch = batchc
                mres = []
                for i, (pepres_mut, pepres) in enumerate(zip(pdb_seq, wt_seq)): # Get mutant residues
                    if pepres_mut != pepres:
                        mres.append(i)
                # print(pdb_seq)
                # print(wt_seq)
                # print(mres)

                batch[0]['string_sequence'][0] = pdb_seq

                # Calculate RLA score
                output_dict = feature_getter(trained_model, tokenizer, batch, focus=True, top_k=top_k,
                                             mres=mres, add_ends=add_ends, ends_k=ends_k, seq_only=seq_only,
                                             dist_threshold=dist_threshold)
                text_feat = output_dict['text']
                focus_mask = output_dict['focus_mask'].to(device=text_feat.device)
                text_mask = output_dict['seq_mask_no_burn_in'] * focus_mask
                if not seq_only:
                    gnn_feat = output_dict['gnn'][:, :(text_feat.shape[1])] # Remove tail padding
                    if text_feat.shape != gnn_feat.shape: # Ensure matching sequence and structure for mutant
                        skip_prot = True
                        print('shape =/= benign')
                        break

                    # print("text_feat", text_feat.shape)
                    # print("gnn_feat", gnn_feat.shape)
                    if score_type == 'COS':
                        scores = (text_feat.unsqueeze(2) @ gnn_feat.unsqueeze(-1)).squeeze(-1).squeeze(-1)
                    elif score_type == 'EUC':
                        diff = text_feat - gnn_feat
                        squared_diff = diff ** 2
                        sum_squared_diff = torch.sum(squared_diff, dim=2)
                        scores = torch.sqrt(sum_squared_diff).squeeze(0)  # [49]

                    scores_mean = (scores * text_mask.float()).sum(1)/text_mask.sum(1)
                else:
                    scores = torch.tensor([np.nan])
                    scores_mean = torch.tensor([0])[0]

                # Save info
                all_seq_scores['aa_seq'].append(pdb_seq)
                all_seq_scores['RLA res scores'].append([score.cpu().numpy() for score in scores])
                all_mean_scores['aa_seq'].append(pdb_seq)
                all_mean_scores['RLA scores'].append(scores_mean.item())


                # print("text_feat", text_feat.shape)
                # print("text_feat_base", text_feat_base.shape)

                if score_type == 'COS':
                    esm_rla_scores = (text_feat.unsqueeze(2) @ text_feat_base.unsqueeze(-1)).squeeze(-1).squeeze(-1).squeeze(0)
                elif score_type == 'EUC':
                    diff = text_feat - text_feat_base
                    squared_diff = diff ** 2
                    sum_squared_diff = torch.sum(squared_diff, dim=2)
                    esm_rla_scores = torch.sqrt(sum_squared_diff).squeeze(0)  # [49]

                esm_rla_scores = (esm_rla_scores * text_mask.float()).sum(1)/text_mask.sum(1)
                esm_rla_scores_muts['aa_seq'].append(pdb_seq)
                esm_rla_scores_muts['ESM-RLA scores'].append(esm_rla_scores.item())


                inds_list, mut_list = get_muts(wt_seq, pdb_seq)
                inds.append(inds_list)
                muts.append(mut_list)
                i_seq += 1
                filt_pdb_seqs.append(pdb_seq)
    pdb_i+=1
    print(pdb_i, pdb, i)
    if pdb not in yichuan_list:
      yichuan_list.append(pdb)
      yichuan_count += 1

    all_scores_muts[pdb] = all_mean_scores
    all_scores_esm_rla_muts[pdb] = esm_rla_scores_muts

# if len(test_subset)==0:
output_results(all_scores_esm_rla_muts,os.path.join("/content/rla/rla_predict.txt"))

{'max_coords_len': 2000, 'shuffle_coords': False, 'max_seq_len': 1024, 'pos_offset': 128, 'burn_in': 0, 'k_neighbors': 30, 'crop_type': 'absolute', 'shuffle_chains': False, 'num_mutations': -1, 'masked_rate': -1.0, 'masked_mode': 'MASK'}
added select filtering... 30
0 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/content/rla/terminator/models/layers/graph_features.py:244: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  n_1 = F.normalize(torch.cross(N_CA, CA_C), dim=-1)
100%|██████████| 2771/2771 [01:54<00:00, 24.21it/s]


1 1PV0 43


100%|██████████| 2569/2569 [01:46<00:00, 24.04it/s]


2 1YU5 64


...

296 HHH_rd2_0155 42


100%|██████████| 802/802 [00:33<00:00, 24.30it/s]


297 EEHEE_rd3_1498 42


100%|██████████| 731/731 [00:30<00:00, 24.24it/s]


298 EHEE_rd2_0372 39


In [ ]:
yichuan_count

298

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

output_file_path = '/content/rla/correlation_results.txt'
output_image_dir = '/content/rla/images/'  # Specify the directory to save images

with open(output_file_path, 'w') as file:
    for pdb in all_scores_muts.keys():
        pdb_dataset = stability_scores_drop[stability_scores_drop['WT_name'] == pdb + '.pdb']
        rla_df = pd.DataFrame(all_scores_muts[pdb])
        esm_rla_df = pd.DataFrame(all_scores_esm_rla_muts[pdb])
        results_merge = rla_df.merge(esm_rla_df).merge(pdb_dataset)

        rla_corr = results_merge['dG_ML'].corr(results_merge['RLA scores'])
        esm_rla_corr = results_merge['dG_ML'].corr(results_merge['ESM-RLA scores'])

        log_message_rla = f'{pdb} RLA PCC: {round(rla_corr, 3)}'
        log_message_esm_rla = f'{pdb} ESM_RLA PCC: {round(esm_rla_corr, 3)}'
        file.write(log_message_rla + '\n')
        file.write(log_message_esm_rla + '\n')

        print(log_message_rla)
        print(log_message_esm_rla)

        fig = plt.figure()
        plt.scatter(results_merge['dG_ML'], results_merge['RLA scores'], s=1)
        plt.xlabel('Observed')
        plt.ylabel('Predicted (RLA scores)')
        plt.title(log_message_rla)
        # plt.savefig(output_image_dir + f'{pdb}_RLA_plot.png')  # Save the plot as a PNG file
        plt.show()

        fig = plt.figure()
        plt.scatter(results_merge['dG_ML'], results_merge['ESM-RLA scores'], s=1)
        plt.xlabel('Observed')
        plt.ylabel('Predicted (ESM-RLA scores)')
        plt.title(log_message_esm_rla)
        plt.savefig(output_image_dir + f'{pdb}_ESM_RLA_plot.png')  # Save the plot as a PNG file
        plt.show()


In [ ]:
!cp /content/rla/rla_predict.txt /content/drive/MyDrive/RLA_results_euclidean

In [ ]:
!cp /content/rla/correlation_results.txt /content/drive/MyDrive/RLA_results_euclidean

In [ ]:
!cp /content/rla/images -r /content/drive/MyDrive/RLA_results_euclidean

In [ ]:
import time

time.sleep(300)

In [ ]:
from google.colab import runtime
runtime.unassign()